In [1]:
!pip install datasets gensim spacy
!python -m spacy download ro_core_news_sm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.6/27.6 MB 25.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 13.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 24.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.8 MB/s eta 0:00:00a 0:00:01
  

In [6]:
# from datasets import load_dataset
# import spacy
# from tqdm.auto import tqdm

# dataset = load_dataset("andyP/ro-paraphrase-bible")
# nlp = spacy.load("ro_core_news_sm", disable=['ner', 'parser'])

# text_columns = [col for col in dataset['train'].column_names if dataset['train'].features[col].dtype == 'string']
# col1, col2 = text_columns[0], text_columns[1]

# list1 = [s for s in tqdm(dataset['train'][col1], desc="Loading Column 1") if s is not None]
# list2 = [s for s in tqdm(dataset['train'][col2], desc="Loading Column 2") if s is not None]

# all_sentences = set(list1 + list2)
# original_sentences_list = list(all_sentences)

def preprocess(text):
    doc = nlp(str(text))
    return [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space]

# processed_corpus = [preprocess(sent) for sent in tqdm(original_sentences_list, desc="Tokenizing Romanian Text")]

# print(f"Finished processing {len(processed_corpus)} sentences.")

In [ ]:
from datasets import load_dataset
import spacy
from tqdm.auto import tqdm

dataset = load_dataset("andyP/ro-paraphrase-bible")
nlp = spacy.load("ro_core_news_sm", disable=['ner', 'parser'])

text_columns = [col for col in dataset['train'].column_names if dataset['train'].features[col].dtype == 'string']
col1, col2 = text_columns[0], text_columns[1]

list1 = [s for s in tqdm(dataset['train'][col1], desc="Loading Column 1") if s is not None]
list2 = [s for s in tqdm(dataset['train'][col2], desc="Loading Column 2") if s is not None]

all_sentences = set(list1 + list2)
original_sentences_list = list(all_sentences)

processed_corpus = []

for doc in tqdm(nlp.pipe(original_sentences_list, batch_size=3000, n_process=-1),
                total=len(original_sentences_list),
                desc="Parallel Tokenizing"):
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space]
    processed_corpus.append(tokens)

print(f"Finished processing {len(processed_corpus)} sentences.")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 1128927 examples [00:06, 164235.60 examples/s]
Parallel Tokenizing: 100%|██████████| 247023/247023 [04:22<00:00, 942.54it/s] 

Finished processing 247023 sentences.


In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    sentences=processed_corpus,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4
)

w2v_model.train(processed_corpus, total_examples=len(processed_corpus), epochs=10)
w2v_model.save("bible_ro_word2vec.model")

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(processed_corpus)]

d2v_model = Doc2Vec(
    vector_size=100,
    window=5,
    min_count=2,
    dm=1,
    epochs=20,
    workers=4
)

d2v_model.build_vocab(tagged_data)
d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)
d2v_model.save("bible_ro_doc2vec.model")

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


NameError: name 'preprocess' is not defined

Word not found in dictionary. Try a word that appears at least twice in the text.


In [22]:
test_tokens = preprocess("Cuvantul este adevar")
inferred_vector = d2v_model.infer_vector(test_tokens)
sims = d2v_model.dv.most_similar([inferred_vector], topn=10)

for tag, similarity in sims:
    print(original_sentences_list[int(tag)], similarity)

Când se întoarse înapoi, Ahitofel l-a sfătuit:„Eu cred că este potrivit, Să intri-acum, în casa-n care Se află-acele țiitoare, Pe cari, de pază le-a lăsat David, atunci când a plecat. Du-te la ele și să știi Că vei ajunge ca să fii Urât, de tatăl tău. Astfel, Va ști întregul Israel Că ești urât de al tău tată, Și se vor întări, de-ndată, Brațele celor ce-ți slujesc. Așa să faci, te sfătuiesc! ” 0.3667803108692169
Iubirea – vă spun lămurit – Nu stă în faptul c-am iubit – Noi, oamenii – pe Dumnezeu, Ci în aceea că, mereu, El ne-a iubit, pe noi, și-a dat, Drept jertfă pentru-al nost’ păcat, Pe Fiul Său, de-a ispășit Ceea ce noi am săvârșit. 0.34932148456573486
Tatăl și-a dat seama atunci că la ceasul acela Isus îi spusese: „Fiul tău trăiește!” Și a crezut el și toată casa lui. 0.34925416111946106
Eu vă spun că le va face răzbunare în grabă. Însă când va veni Fiul omului, va găsi el oare credință pe pământ? 0.33917734026908875
„Știi tu când capra pui-și scoate? Hai spune, de le știi pe toa